# Access Review

Reproduces the access review demo for the pizza shop (POS/CRM). 

**Objectives**
- Join `users.csv`, `roles.csv`, `assignments.csv`
- Flag orphan/inactive users with access
- Flag high-privilege users (>=8)
- Save `outputs/findings.csv` and show KPIs

In [3]:
from pathlib import Path
import pandas as pd
base_dir = Path('..').resolve()

In [7]:
assignments = pd.read_csv(base_dir/'data'/'assignments.csv')
roles = pd.read_csv(base_dir/'data'/'roles.csv')
users = pd.read_csv(base_dir/'data'/'users.csv')

assignments.head(), users.head(), roles.head()

(    uid       role
 0  u001  POS_Admin
 1  u001   CRM_User
 2  u002   CRM_User
 3  u003   POS_User
 4  u004  CRM_Admin,
     uid   name      dept  active
 0  u001  Alice       Ops    True
 1  u002    Bob  Delivery   False
 2  u003  Carol     Sales    True
 3  u004   Dave       Ops    True,
         role  priv_level
 0  POS_Admin           9
 1   POS_User           3
 2  CRM_Admin           9
 3   CRM_User           3)

In [8]:
merged = assignments.merge(users, on='uid', how='left').merge(roles, on='role', how='left')
merged.head()

,uid,role,name,dept,active,priv_level
0,u001,POS_Admin,Alice,Ops,True,9
1,u001,CRM_User,Alice,Ops,True,3
2,u002,CRM_User,Bob,Delivery,False,3
3,u003,POS_User,Carol,Sales,True,3
4,u004,CRM_Admin,Dave,Ops,True,9


In [ ]:
orphan = merged[merged['active'].fillna(False)==False][['uid','role']].drop_duplicates()
orphan['issue'] = 'orphan_or_inactive_user_with_access'
orphan['severity'] = 'High'

high = merged[merged['priv_level'].fillna(0)>=8][['uid','role','priv_level']].drop_duplicates()
high['issue'] = 'high_privilege_review_required'
high['severity'] = 'Medium'

findings = pd.concat([orphan, high], ignore_index=True)
findings

,uid,role,issue,severity,priv_level
0,u002,CRM_User,orphan_or_inactive_user_with_access,High,NaN
1,u001,POS_Admin,high_privilege_review_required,Medium,9.0
2,u004,CRM_Admin,high_privilege_review_required,Medium,9.0


In [12]:
kpi_orphans = len(orphan)
kpi_high = len(high)
{'orphans': kpi_orphans, 'high_privilege': kpi_high, 'total_findings': len(findings)}

{'orphans': 1, 'high_privilege': 2, 'total_findings': 3}

In [13]:
out = base_dir/'outputs'
out.mkdir(parents=True, exist_ok=True)
findings.to_csv(out/'findings.csv', index=False)
print('Saved:', out/'findings.csv')

Saved: C:\Project\GRC 1\01_Case\A_Access_Review\outputs\findings.csv


In [16]:
from datetime import datetime
import hashlib
p = out/'findings.csv'
sha = hashlib.sha256(p.read_bytes()).hexdigest()
ts = datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')
print(f'{ts}  {p.name}  SHA256={sha}')

2025-09-10T22:58:51Z  findings.csv  SHA256=426790cb24b149c700ff591363a73f9bcaad7272c1c9d2be50ab77c941f554fb
